In [1]:
# # package requirements
#!pip install geemap
#!pip install ipyleaflet
#!pip install tkinter
#!pip install tkcalendar
# # requirement to launch geemap in Colab
# !pip install tornado==5.1.1 
# ee.Authenticate()
# ee.Initialize()


# import packages
import ee
import ee; ee.Initialize()
import geemap
import os
import ipyleaflet
import numpy as np
import tkinter as tk 
from tkinter import ttk
from tkcalendar import DateEntry 

In [16]:
## Functions

# define intersect function for intersected county and states
def intersect(featurecollection):
  return featurecollection.intersection(cntyGeom, 1)
def addArea(feature):
  return feature.set({'areaHa': feature.geometry().area().divide(100 * 100)})

#Define  function for NDVI & NDWI
def addIndices(image):
    # Create the NDVI and NDWI spectral indices.
    if 'R' == image.bandNames().getInfo()[0]:
        ndvi = image.normalizedDifference(['N', 'R'])
        ndwi = image.normalizedDifference(['G', 'N'])
        
    if 'SR_B5' == image.bandNames().getInfo()[0]:
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4'])
        ndwi = image.normalizedDifference(['SR_B3', 'SR_B5'])
    # Create some binary images from thresholds on the indices.
    
    # This threshold is designed to detect bare land.
    bare = ndvi.lt(0.2).And(ndwi.lt(0.3))

    # Mask and mosaic visualization images.  The last layer is on top.
    mosaic = ee.ImageCollection([
    # NDWI > 0.5 is water.  Visualize it with a blue palette.
    ndwi.updateMask(ndwi.gte(0.65)).visualize(min= 0.5, max= 1, palette= ['00FFFF', '0000FF']),
    # NDVI > 0.2 is vegetation.  Visualize it with a green palette.
    ndvi.updateMask(ndvi.gte(0.1)).visualize(min= -1, max= 1, palette= ['FF0000', '00FF00']),
    # Visualize the other bare areas as gray.
    bare.updateMask(bare).visualize(**{'palette': ['AAAAAA']}),
    ]).mosaic()
    return image.addBands(mosaic).addBands(ndvi)

# Function to Normalize Image
# Pixel Values should be between 0 and 1
# Formula is (x - xmin) / (xmax - xmin)
def normalize(image):
    bandNames = image.bandNames()
  # Compute min and max of the image
    minDict = image.reduceRegion(
    reducer= ee.Reducer.min(),
    geometry= rgnGeom,
    scale= 10,
    maxPixels= 1e20,
    bestEffort= True,
    )
    maxDict = image.reduceRegion(
    reducer= ee.Reducer.max(),
    geometry= rgnGeom,
    scale= 10,
    maxPixels= 1e20,
    bestEffort= True,
    )
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))
    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized

# create a dictionary with the values for every class
def func_hoq(item):  
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('classification')).format('%.0f')
    # The result will be in square meters, this converts them into acres
    area = ee.Number(areaDict.get('sum')).divide(1e6).multiply(247.10538146717).format('%.0f') 
    return ee.List([classNumber, area])

# calculate land area statistics in acres
def stat_area(image):
    areaImage = ee.Image.pixelArea().addBands(image)
    areas = areaImage.reduceRegion(
        reducer= ee.Reducer.sum().group(groupField= 1,groupName= 'classification'),
        geometry= rgnGeom,
        scale= 30,
        maxPixels= 1e20)
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(func_hoq)
    result = ee.Dictionary(classAreaLists.flatten());
    return result.getInfo()

# RUN the model
def model(main_lst,rgnGeom):
    # collect imagery dataset
    if main_lst['data'] == list(datasets.values())[0]:
        collection = ee.ImageCollection('USDA/NAIP/DOQQ')\
                          .filterBounds(rgnGeom) \
                          .filter(ee.Filter.date(main_lst['Start_Date'], main_lst['End_Date'])).median()
    
    if main_lst['data'] == list(datasets.values())[1]:
        collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')\
                          .filterBounds(rgnGeom) \
                          .filter(ee.Filter.date(main_lst['Start_Date'], main_lst['End_Date'])).median()
    
    #USDA NASS Cropland Data Layers for training and clipping by crop type
    USDA = ee.ImageCollection('USDA/NASS/CDL') \
                      .filter(ee.Filter.date(main_lst['Start_Date'], main_lst['End_Date'])).median()

    #JRC water Data Layers for training and validation
    JRC_collection = ee.ImageCollection("JRC/GSW1_4/YearlyHistory") \
                      .filter(ee.Filter.date(main_lst['Start_Date'],main_lst['End_Date'])).median()
    
    #clip the  image to area of study
    image = collection.clip(rgnGeom)

    # create a compopsite of Indices for prediction 
    composite = addIndices(image)

    # Training and validatoin datasets
    JRC = JRC_collection.clip(rgnGeom).select('waterClass')

    # Mask and mosaic training and validatoin datasets
    mosaic = ee.ImageCollection([
        USDA.clip(rgnGeom).select('cultivated').eq(2).visualize(palette= ['00FFFA', '01fffa']),
        JRC.updateMask(JRC.eq(3)).visualize(palette= ['02FFFF']),
        JRC.updateMask(JRC.eq(2)).visualize(palette= ['02FFFF']),
        ]).mosaic().select('vis-red').rename('class')
    
    # filter the values to the cultivated, wetland, and barren classes
    mosaic = mosaic.updateMask(mosaic.expression("b('class') == 0 || b('class') == 1 || b('class') == 2"))

    ## Supervised Classification

    # Make the training dataset.
    points = mosaic.sample(**{
        'region': rgnGeom,
        'scale': 10,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    # Use these bands for prediction.
    bands = [ 'vis-red', 'vis-green', 'vis-blue']

    # This property of the table stores the land cover labels.
    label = 'class'

    # Overlay the points on the imagery to get training
    sample = composite.select(bands).sampleRegions(**{
      'collection': points,
      'properties': [label],
      'scale': 1
    })

    # Adds a column of deterministic pseudorandom numbers. 
    sample = sample.randomColumn()

    # 70% training, 30% validation
    split = 0.7
    training = sample.filter(ee.Filter.lt('random', split)) #Training dataset
    validation = sample.filter(ee.Filter.gte('random', split)) #Validation dataset

    # run the classification model

    # Train a classifier with default parameters.
    if main_lst['method'] == list(methods.values())[0]:
        classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)
    if main_lst['method'] == list(methods.values())[1]:
        classifier = ee.Classifier.smileCart().train(training, label, bands)
    if main_lst['method'] == list(methods.values())[2]:
        classifier = ee.Classifier.libsvm().train(training, label, bands)    
    if main_lst['method'] == list(methods.values())[3]:    
        classifier = ee.Classifier.smileNaiveBayes().train(training, label, bands)

    # Classify the image with the same bands used for training.
    classified = composite.select(bands).classify(classifier)

    #Apply Normalize function
    normalizedImage = normalize(classified )

    #Accuracy Assessment
    validated = validation.classify(classifier)
    ConfusionMatrix = validated.errorMatrix('class', 'classification')

    # Flooded Field Extraction

    # total cropland
    Totalcropland = USDA.select('cultivated').clip(rgnGeom)
    Totalcropland = Totalcropland.eq(2)
    cultivated = Totalcropland.mask(Totalcropland.eq(1))
    #statistics
    cultivated_total_stats = stat_area(cultivated)

    # Flooded Fields
    Flooded = classified.mask(cultivated)
    FloodedField_Total = Flooded.mask(Flooded.eq(0).Or(Flooded.eq(2)))   

    #statistics
    Flooded_total_stats = stat_area(FloodedField_Total)

    # Flooded Fields by crop type
    croplands = USDA.clip(rgnGeom).select('cropland')
    cropland_total_stats = stat_area(croplands)

    # statistics for selected crop type
    cropland_crop = ee.Image(croplands).eq(list(crop_dict.values())[int(main_lst['crop_type'])])
    cultivated_crop = cropland_crop.mask(cropland_crop.eq(1))
    # Crop Flooded Fields
    Flooded_crop = normalizedImage.mask(cultivated_crop)
    FloodedField_crop = Flooded_crop.mask(Flooded_crop.eq(1)) 
    # USDA cultivated statistics
    cultivated_crop_stats = stat_area(cultivated_crop)
    # FloodedField statistics
    Flooded_crop_stats = stat_area(FloodedField_crop)

    #selected crop
    crop = list(crop_dict.keys())[int(main_lst['crop_type'])]


    #skip non crop masks
    skip=['81','82','83','87','88','92','111','112','121','122','123','124','131','141','142','143','152','176','190','195']
    
    # make a list of statistics
    stats = [{'cultivated_total_stats':cultivated_total_stats},{'Flooded_total_stats':Flooded_total_stats},
             {'cultivated %s stats'%crop :cultivated_crop_stats },
             {'Flooded %s stats'%crop:Flooded_crop_stats}]

    for i in cropland_total_stats.keys():
        if i not in skip:
            # crop mask
            cropland_crop = ee.Image(croplands).eq(int(i))
            cultivated_crop = cropland_crop.mask(cropland_crop.eq(1))

            # Crop Flooded Fields
            Flooded_crop = normalizedImage.mask(cultivated_crop)
            FloodedField_crop = Flooded_crop.mask(Flooded_crop.eq(1)) 

            # USDA cultivated statistics
            cultivated_crop_stats = stat_area(cultivated_crop)

            # FloodedField statistics
            Flooded_crop_stats = stat_area(FloodedField_crop)

            # append crop stats
            stats.append({'cultivated %s stats (in acres)' % list(crop_dict.keys())[list(crop_dict.values()).index(int(i))] : cultivated_crop_stats,
                          'Flooded %s stats (in acres)' % list(crop_dict.keys())[list(crop_dict.values()).index(int(i))] : Flooded_crop_stats})
    #**************************************************************************
    # Exporting Results
    #**************************************************************************

    # Create a Feature with {} geometry and the value we want to export.
    # Use .array() to convert Confusion Matrix to an Array so it can be
    # exported in a CSV file
    # Create a Feature with {} geometry and the sum of Flooded pixels value we want to export. 

    feature = ee.FeatureCollection([
      ee.Feature(None, {
        'Cultivated':stats[0]['cultivated_total_stats']['1'],
        'Barren': stats[1]['Flooded_total_stats']['0'],
        'Flooded' :stats[1]['Flooded_total_stats']['2'],
        'cultivated %s stats'%crop:stats[2]['cultivated %s stats'%crop],
        'flooded %s stats'%crop:stats[3]['Flooded %s stats'%crop],
        'crop_statistics': stats[4:],
        'classsifcation_accuracy': ConfusionMatrix.accuracy(),
        'classsifcation_matrix': ConfusionMatrix.array(),
        'classsifcation_kappa' : ConfusionMatrix.kappa().getInfo(),
        'classsifcation_producersAccuracy': ConfusionMatrix.producersAccuracy().getInfo(),
        'classsifcation_consumersAccuracy': ConfusionMatrix.consumersAccuracy().getInfo()
      })
      ])
    # Exporting Results to Drive
    task_config = {
        'folder': 'GEE/Fall', # output Google Drive folder
        'fileFormat': 'csv',  
        }


    task = ee.batch.Export.table.toDrive(feature, main_lst['outfilename'], **task_config)
    task.start()

    #Export Geotiff to Drive

    geemap.ee_export_image_to_drive(
        ee_object=Flooded.clip(rgnGeom),
        description= main_lst['description'],
        folder='GEE/Fall',
        region=None,
        scale=1,
        max_pixels=1.0e13,
        file_format="GEO_TIFF",
        format_options={},
    )

    ## Download to local Desktop only if the size of study area is small
    # out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    # out_file = os.path.join(out_dir, 'cluster22.tif')
    # geemap.ee_export_image(Flooded.clip(rgnGeom), filename=out_file, scale=30)

    return [stats,Flooded, ConfusionMatrix] # Final Outputs

In [10]:
# Initital variables
# manually defining area of study, dates, and crop type
main_lst={'State_Name':'Minnesota', 'County_Name':'Jackson', 'Draw':'1','Start_Date':'2019-01-01','End_Date':'2020-01-01', 
          'data':'0','crop_type':'1','method':'2','outfilename':'statistics_Murray_2019' ,'description':'Supervised-Murray_Jackson_2019'}
#crop types
crop_dict={"Corn":1,"Cotton":2,"Rice":3,"Sorghum":4,"Soybeans":5,"Sunflower":6,"Peanuts":10,"Tobacco":11,"Sweet Corn":12,
    "Pop or Orn Corn":13,"Mint":14,"Barley":21,"Durum Wheat":22,"Spring Wheat":23,"Winter Wheat":24,"Other Small Grains":25,
    "Dbl Crop WinWht/Soybeans":26,"Rye":27,"Oats":28,"Millet":29,"Speltz":30,"Canola":31,"Flaxseed":32,"Safflower":33,
    "Rape Seed":34,"Mustard":35,"Alfalfa":36,"Other Hay/Non Alfalfa":37,"Camelina":38,"Buckwheat":39,"Sugarbeets":41,
    "Dry Beans":42,"Potatoes":43,"Other Crops":44,"Sugarcane":45,"Sweet Potatoes":46,"Misc Vegs & Fruits":47,"Watermelons":48,
    "Onions":49,"Cucumbers":50,"Chick Peas":51,"Lentils":52,"Peas":53,"Tomatoes":54,"Caneberries":55,"Hops":56,"Herbs":57,
    "Clover/Wildflowers":58,"Sod/Grass Seed":59,"Switchgrass":60,"Fallow/Idle Cropland":61,"Forest":63,"Shrubland":64,"Barren":65,
    "Cherries":66,"Peaches":67,"Apples":68,"Grapes":69,"Christmas Trees":70,"Other Tree Crops":71,"Citrus":72,"Pecans":74,"Almonds":75,
    "Walnuts":76,"Pears":77,"Clouds/No Data":81,"Developed":82,"Water":83,"Wetlands":87,"Nonag/Undefined":88,"Aquaculture":92,
    "Open Water":111,"Perennial Ice/Snow":112,"Developed/Open Space":121,"Developed/Low Intensity":122,"Developed/Med Intensity":123,
    "Developed/High Intensity":124,"Barren":131,"Deciduous Forest":141,"Evergreen Forest":142,"Mixed Forest":143,"Shrubland":152,
    "Grassland/Pasture":176,"Woody Wetlands":190,"Herbaceous Wetlands":195,"Pistachios":204,"Triticale":205,"Carrots":206,
    "Asparagus":207,"Garlic":208,"Cantaloupes":209,"Prunes":210,"Olives":211,"Oranges":212,"Honeydew Melons":213,"Broccoli":214,
    "Avocados":215,"Peppers":216,"Pomegranates":217,"Nectarines":218,"Greens":219,"Plums":220,"Strawberries":221,"Squash":222,
    "Apricots":223,"Vetch":224,"Dbl Crop WinWht/Corn":225,"Dbl Crop Oats/Corn":226,"Lettuce":227,"Dbl Crop Triticale/Corn":228,
    "Pumpkins":229,"Dbl Crop Lettuce/Durum Wht":230,"Dbl Crop Lettuce/Cantaloupe":231,"Dbl Crop Lettuce/Cotton":232,"Dbl Crop Lettuce/Barley":233,
    "Dbl Crop Durum Wht/Sorghum":234,"Dbl Crop Barley/Sorghum":235,"Dbl Crop WinWht/Sorghum":236,"Dbl Crop Barley/Corn":237,
    "Dbl Crop WinWht/Cotton":238,"Dbl Crop Soybeans/Cotton":239,"Dbl Crop Soybeans/Oats":240,"Dbl Crop Corn/Soybeans":241,
    "Blueberries":242,"Cabbage":243,"Cauliflower":244,"Celery":245,"Radishes":246,"Turnips":247,"Eggplants":248,"Gourds":249,
    "Cranberries":250,"Dbl Crop Barley/Soybeans":254}

#classification methods
methods={'Random Forest':'0','CART':'1','SVM':'2','Naive Bayes':'3'}

# datasets
datasets ={'NAIP 1 meter resolution':'0','Landsat 9 30 meter resolution':'1'}

In [4]:
# Create a tkinter window to get user inpupts or use next cell for default values
window = tk.Tk()
window.title('Input config')
window.geometry("650x800")

# label text for title
ttk.Label(window, text = "Fooded fields modeling system configuration", 
          background = 'green', foreground ="white", 
          font = ("Times New Roman", 15)).grid(row = 0, column = 2)


# Define area of study and dates

# Define entry text and date values
text1 = tk.Entry(window)
text1.grid(row=1,column=2,padx=20,pady=30)

text2 = tk.Entry(window)
text2.grid(row=2,column=2,padx=20,pady=30)

cal1=DateEntry(window,selectmode='day')
cal1.grid(row=4,column=2,padx=20,pady=30)

cal2=DateEntry(window,selectmode='day')
cal2.grid(row=5,column=2,padx=20,pady=30)

# Define entry text and date labels
l1=tk.Label(window,text='State :',bg='yellow')  # Label to display state 
l1.grid(row=1,column=1)

l2=tk.Label(window,text='County :',bg='yellow')  # Label to display county 
l2.grid(row=2,column=1)

l3=tk.Label(window,text='Start date :',bg='yellow')  # Label to display date 
l3.grid(row=4,column=1)

l4=tk.Label(window,text='End date :',bg='yellow')  # Label to display date 
l4.grid(row=5,column=1)



# label the drawing combobox
ttk.Label(window, text = "Drawing area of interest manually?",
          font = ("Times New Roman", 10)).grid(column = 1, row = 3, padx = 10, pady = 25)

# create the combobox 
n = tk.StringVar()
drawchosen = ttk.Combobox(window, width = 27, textvariable = n)
  
# Adding combobox drop down list
drawchosen['values'] = ['No','Yes']
drawchosen.grid(column = 2, row = 3)
drawchosen.current()

# label the datasets combobox
ttk.Label(window, text = "Select the dataset:",
          font = ("Times New Roman", 10)).grid(column = 1, row = 6, padx = 10, pady = 25)

# create the combobox 
n = tk.StringVar()
datachosen = ttk.Combobox(window, width = 27, textvariable = n)
  
# Adding combobox drop down list
datachosen['values'] = list(datasets.keys())
datachosen.grid(column = 2, row = 6)
datachosen.current()

# label the crop type combobox
ttk.Label(window, text = "Select the crop type :",
          font = ("Times New Roman", 10)).grid(column = 1, row = 7, padx = 10, pady = 25)

# create the combobox 
n = tk.StringVar()
cropchosen = ttk.Combobox(window, width = 27, textvariable = n)
  
# Adding combobox drop down list
cropchosen['values'] = list(crop_dict.keys())
cropchosen.grid(column = 2, row = 7)
cropchosen.current()

# label the method combobox
ttk.Label(window, text = "Select the classification method :",
          font = ("Times New Roman", 10)).grid(column = 1, row = 8, padx = 10, pady = 25)

# create the combobox 
n = tk.StringVar()
methodchosen = ttk.Combobox(window, width = 27, textvariable = n)
  
# Adding combobox drop down list
methodchosen['values'] = list(methods.keys())
methodchosen.grid(column = 2, row = 8)
methodchosen.current()

# Define names of final files for saving in google drive
text3 = tk.Entry(window)
text3.grid(row=9,column=2,padx=20,pady=30)

text4 = tk.Entry(window)
text4.grid(row=10,column=2,padx=20,pady=30)

# Define entry text and date labels
l3=tk.Label(window,text='file name to save statistcs in Google Drive:',bg='yellow')  # Label to display state 
l3.grid(row=9,column=1)

l4=tk.Label(window,text='file name to save map in Google Drive:',bg='yellow')  # Label to display county 
l4.grid(row=10,column=1)

# save inputs in a dictionary
def my_upd(): # triggered on Button Click
    lst=[str(text1.get()), str(text2.get()), str(drawchosen.current()), str(cal1.get_date()),str(cal2.get_date()),
        str(datachosen.current()), str(cropchosen.current()),str(methodchosen.current()), str(text3.get()), str(text4.get()) ]
    main_lst.update(dict(zip(main_lst, lst)))
    return
   
# on Button click "Confirm" to record study area and date selection
b1=tk.Button(window,text='Confirm', command=lambda:my_upd())
b1.grid(row=8,column=3)

# Create an Exit button.
b2 =tk.Button(window, text = "Exit",  command = window.destroy)
b2.grid(row=9,column=3)

# Execute tkinter
window.mainloop()

print(main_lst)

{'State_Name': 'Minnesota', 'County_Name': 'Jackson', 'Draw': '1', 'Start_Date': '2019-01-01', 'End_Date': '2020-01-01', 'data': '0', 'crop_type': '0', 'method': '2', 'outfilename': 'stats', 'description': 'map'}


In [11]:
#Define area of study

# step 1: select geomety
#state 
stateBoundary = geemap.ee.FeatureCollection('TIGER/2018/States').select('NAME')\
.filter(geemap.ee.Filter.eq('NAME', main_lst['State_Name']))

#county 
CountyBoundary = geemap.ee.FeatureCollection('TIGER/2018/Counties').select('NAME')\
.filter(geemap.ee.Filter.eq('NAME', main_lst['County_Name']))

# geom of area of study
cntyGeom = CountyBoundary.geometry()

#filter geomety
#Apply intersect function
region = stateBoundary.map(intersect)
areaAdded = region.map(addArea)
rgnGeom = areaAdded.geometry() # rgnGeom is final geometry of the study area


In [17]:
# Draw an area of interest only if you selected to define geometry manually
if main_lst['Draw'] == '0':
    %%script false
# Display to draw
Map = geemap.Map()
Map.centerObject(rgnGeom, zoom=10)
Map.addLayer(rgnGeom, {}, 'region')
Map

Map(center=[43.74999404614094, -95.25706699999093], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
# Record the manual geometry of area of interest 
if main_lst['Draw'] == '0':
    %%script false 
# get goemetry of drawing for the area of interest 
roi = ee.FeatureCollection(Map.draw_features)
rgnGeom = roi.geometry()  # update the drawing to the final geometry of the study area

In [19]:
# Run the model
flooded_model =model(main_lst,rgnGeom)

# show the map
Map = geemap.Map()
Map.centerObject(rgnGeom, zoom=10)
Map.addLayer(flooded_model[1].randomVisualizer(), {}, 'Color Infrared')
Map

Exporting Supervised-Murray_Jackson_2019 ...


Map(center=[43.7490023294005, -95.2556789999998], controls=(WidgetControl(options=['position', 'transparent_bg…